### Calculo de media y varianza del dataset data_ok.csv
Septiembre 2024

v3: Transforma cada celda del dataset en un elemento (j,v),  donde "j" es la columna de la celda y "v" es el valor de la celda del rdd. Resuelve el problema con esta nueva estructura del dataset

Realizado por:
- GEORGELYS MARCANO
- ERICH GONZÁLEZ
- DANIEL GUTIÉRREZ

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import numpy as np
from pyspark import RDD
from typing import Tuple 

# Configuración de la sesión de Spark
session = SparkSession.builder.master('local[*]').getOrCreate()
context = session.sparkContext

FILE_PATH = '../0-SPAI/1-datos/data_ok.csv'
rdd0_raw = context.textFile(FILE_PATH)

def string_to_float(element):
    return np.array([float(x) for x in element.split()])
# Cada elemento del RDD es un numpy array de floats
rdd1_numpys = rdd0_raw.map(string_to_float)

def create_tuples(element):
    # Devuelve una lista de (índice, (valor, 1))
    # El 1 es un para generar un contador del total de elementos
    return [(key, (value, 1)) for key, value in enumerate(element)]

rdd2 = rdd1_numpys.flatMap(create_tuples)

def sum_tuples(tuple1, tuple2):
    value_sum = tuple1[0] + tuple2[0]
    count_sum = tuple1[1] + tuple2[1]
    return (value_sum, count_sum)

# Reduce por clave para obtener (suma de valores, conteo de valores)
rdd3 = rdd2.reduceByKey(sum_tuples)

# Ordenar por clave
rdd4 = rdd3.sortByKey()

def divide_by_num_elements(element):
    key, (value_sum, count) = element
    return (key, value_sum / count)

# Calcular la media para cada índice
means_rdd = rdd4.map(divide_by_num_elements)

# En este caso, como el número de columnas, y por tanto el número de medias
# no es big data (son solo 211 valores) se pueden pasar a la memoria sin 
# colapsarla.
means_list = means_rdd.collect()

# Extraer solo los valores de las medias manteniendo el orden de los índices
# y convertir a numpy array.
means_numpy = np.array([value for _, value in means_list])


# Con los valores de las medias en un numpy array se puede operar sobre
# los valores originales, que también estaban en un numpy.
def squared_diffs(element):
    return (element - means_numpy)**2
rdd5 = rdd1_numpys.map(squared_diffs)

# Aplicar de nuevo la función create_tuples para cada array
rdd6 = rdd5.flatMap(create_tuples)
# Reduce por clave para obtener (suma de valores, conteo de valores)
rdd7 = rdd6.reduceByKey(sum_tuples)
# Ordenar por clave
rdd8 = rdd7.sortByKey()
# Calcular varianza
variance_rdd = rdd8.map(divide_by_num_elements)
# Calcular desvest
desvest_rdd = variance_rdd.map(lambda x: x[1]**(1/2))
desvest_rdd.take(5)

[633.8912890121146,
 56288.3055204925,
 1720.3504419886556,
 2425645.279927015,
 0.0]